In [33]:
#!pip install boto3
#!pip install lz4framed
import os
import io
import yaml
import json
from datetime import datetime, timedelta

import pandas as pd
import psycopg2
import boto3

import requests
from tqdm import tqdm

from dateutil.parser import parse as parse_date
from urllib.parse import urlencode, quote, urlparse

import requests
#import lz4framed
from urllib.parse import urljoin
import gzip
import subprocess
import tarfile

In [41]:
from botocore import UNSIGNED
from botocore.config import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [42]:
CANNED_BASE_URL = 'http://s3.amazonaws.com/ooni-data/canned/'
def load_canned_index(bucket_date):
    r = requests.get(
            urljoin(CANNED_BASE_URL, '{}/index.json.gz'.format(bucket_date))
        )
    blob = r.content
    return gzip.decompress(blob).split(b'\n')

In [45]:
def decompress_tar(filename):
    tar_data = b''
    p = subprocess.Popen(["lz4", "-d"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    r = requests.get(urljoin(CANNED_BASE_URL, filename))
    stdout_data, _ = p.communicate(input=r.content)
    return tarfile.open(mode="r:", fileobj=io.BytesIO(stdout_data))

canned_index = load_canned_index('2020-03-01')
  ## XXX need to filter by test_name
can = json.loads(canned_index[3])
tarfd = decompress_tar(can['filename'])
print(tarfd.getmembers())

[<TarInfo '2020-03-01/20190510T040337Z-US-AS11426-facebook_messenger-20200301T024545Z_AS11426_54ojljTRnjC1vCLE6pvP4p8bbCazhtyOeT81GFokyec7l6g4lE-0.2.0-probe.json' at 0x7fd17409b600>, <TarInfo '2020-03-01/20200229T193728Z-RU-AS48190-facebook_messenger-20200301T165642Z_AS48190_3pZTXbZ6sTYJDmFFkITEjMvPxK0WvrkB8sZ5U9UcYFs1dPcGN1-0.2.0-probe.json' at 0x7fd175c247a0>, <TarInfo '2020-03-01/20200229T195919Z-RU-AS48190-facebook_messenger-20200301T165632Z_AS48190_k7dMoI2IzaLSCkyxaFNtt293cexYP8bEQ1d6YvV7moupO6Ogz4-0.2.0-probe.json' at 0x7fd175c24bb0>, <TarInfo '2020-03-01/20200229T235137Z-US-AS11426-facebook_messenger-20200229T235137Z_AS11426_zA45RhkyebVlBWpO2ECJHm5m0XrMRc6wyZihW7lENE5nZeT9tC-0.2.0-probe.json' at 0x7fd175c24ef0>, <TarInfo '2020-03-01/20200301T000013Z-TW-AS9916-facebook_messenger-20200301T000013Z_AS9916_efcQo40WrpYmrywzfJz2uwFT6tNn4mc1FU4NyswkkooL4pkSJy-0.2.0-probe.json' at 0x7fd175c24870>, <TarInfo '2020-03-01/20200301T000023Z-RU-AS41661-facebook_messenger-20200301T000026Z_AS4166